In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving destinations_db.csv to destinations_db.csv


In [ ]:
df = pd.read_csv("destinations_db.csv")
df.head()

,id,place_name,state,city,city_tag,phone,category,description,image_url,reviews_count,rating,lat,lng
0,1,Taman Mini Indonesia Indah,Daerah Khusus Ibukota Jakarta,Kota Jakarta Timur,jakarta,+62 804 178 9789,Wisata Rekreasi dan Keluarga,Taman yang memamerkan budaya dan alam Indonesi...,https://storage.googleapis.com/travelease-buck...,164271,4.6,-6.302039,106.889892
1,2,Taman Nasional Kepulauan Seribu,Daerah Khusus Ibukota Jakarta,Kab. Administrasi Kepulauan Seribu,jakarta,-,Wisata Alam,"Penyu, snorkeling dan menyelam di taman alam y...",https://storage.googleapis.com/travelease-buck...,555,4.6,-5.745366,106.615182
2,3,Pantai Pasir Perawan,Daerah Khusus Ibukota Jakarta,Kab. Administrasi Kepulauan Seribu,jakarta,+62 857-1048-1038,Wisata Pantai,"Di pesisir pantai, terdapat hutan mangrove hij...",https://storage.googleapis.com/travelease-buck...,717,4.8,-5.854806,106.622534
3,4,Wisata pulau,Daerah Khusus Ibukota Jakarta,Kab. Administrasi Kepulauan Seribu,jakarta,+62 858-8930-2170,Wisata Pulau,"Pantai pulau simpel dengan air jernih, serta k...",https://storage.googleapis.com/travelease-buck...,441,4.6,-5.934315,106.587570
4,5,Wisata pantai. Pulau pari,Daerah Khusus Ibukota Jakarta,Kab. Administrasi Kepulauan Seribu,jakarta,+62 858-8930-7268,Wisata Pantai,Pulau ini menawarkan pesona pantai tropis yang...,https://storage.googleapis.com/travelease-buck...,862,4.6,-5.862751,106.612058


In [ ]:
# Contoh data kategori dan lokasi
category = df['category'].tolist()
states =  df['state'].tolist()
cities =  df['city'].tolist()
city_tag =  df['city_tag'].tolist()
rating = df['rating'].tolist()
reviews_count = df['reviews_count'].tolist()

In [ ]:
# Encode kategori tempat wisata dan lokasi
category_encoder = LabelEncoder()
state_encoder = LabelEncoder()
city_encoder = LabelEncoder()
city_tag_encoder = LabelEncoder()
rating_encoder = LabelEncoder()
reviewscount_encoder = LabelEncoder()

In [ ]:
category_ids = category_encoder.fit_transform(category)
state_ids = state_encoder.fit_transform(states)
city_ids = city_encoder.fit_transform(cities)
city_tag_ids = city_tag_encoder.fit_transform(city_tag)
rating_ids = rating_encoder.fit_transform(rating)
reviewscount_ids = reviewscount_encoder.fit_transform(reviews_count)

In [ ]:
# Input layers
category_input = Input(shape=(1,), name='category_input')
state_input = Input(shape=(1,), name='state_input')
city_input = Input(shape=(1,), name='city_input')
city_tag_input = Input(shape=(1,), name='city_tag_input')
rating_input = Input(shape=(1,), name='rating_input')
reviews_count_input = Input(shape=(1,), name='reviews_count_input')

In [ ]:
# Embedding layers untuk kategori, state, dan city
category_embedding = Embedding(input_dim=len(category_encoder.classes_), output_dim=8)(category_input)
state_embedding = Embedding(input_dim=len(state_encoder.classes_), output_dim=8)(state_input)
city_embedding = Embedding(input_dim=len(city_encoder.classes_), output_dim=8)(city_input)
city_tag_embedding = Embedding(input_dim=len(city_tag_encoder.classes_), output_dim=8)(city_tag_input)
rating_embedding = Embedding(input_dim=len(rating_encoder.classes_), output_dim=8)(rating_input)
reviews_count_embedding = Embedding(input_dim=len(reviewscount_encoder.classes_), output_dim=8)(reviews_count_input)

In [ ]:
# Flatten embeddings
category_flat = Flatten()(category_embedding)
state_flat = Flatten()(state_embedding)
city_flat = Flatten()(city_embedding)
city_tag_flat = Flatten()(city_tag_embedding)
rating_flat = Flatten()(rating_embedding)
reviews_count_flat = Flatten()(reviews_count_embedding)

In [ ]:
# Concatenate semua fitur
concat = Concatenate()([category_flat, state_flat, city_flat, city_tag_flat, rating_flat, reviews_count_flat])

In [ ]:
# Dense layers untuk menghasilkan rekomendasi
dense_1 = Dense(32, activation='relu')(concat)
dense_2 = Dense(16, activation='relu')(dense_1)
output = Dense(8, activation='relu')(dense_2)

In [ ]:
# Buat model
model = Model(inputs=[category_input, state_input, city_input, city_tag_input, rating_input, reviews_count_input], outputs=output)
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Data training
X_train = [np.array(category_ids), np.array(state_ids), np.array(city_ids), np.array(city_tag_ids), np.array(rating_ids), np.array(reviewscount_ids)]
y_train = np.random.rand(len(category_ids), 8)

In [ ]:
# Training model
model.fit(X_train, y_train, epochs=20, batch_size=1, validation_split=0.2)


Epoch 1/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1989 - val_loss: 0.1247
Epoch 2/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1070 - val_loss: 0.1212
Epoch 3/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1015 - val_loss: 0.0874
Epoch 4/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0625 - val_loss: 0.0887
Epoch 5/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0533 - val_loss: 0.0907
Epoch 6/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0454 - val_loss: 0.0930
Epoch 7/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0403 - val_loss: 0.0949
Epoch 8/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0375 - val_loss: 0.0975
Epoch 9/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0340 - val_loss: 0.0973
Epoch 10/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0328 - val_loss: 0.0960
Epoch 11/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0296 - val_loss: 0.0993
Epoch 12/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

In [ ]:
# Menghasilkan vektor embedding untuk item rekomendasi
item_embeddings = model.predict(X_train)
print("Embedding untuk tiap item:\n", item_embeddings)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Embedding untuk tiap item:
 [[0.6119567  0.49610943 0.85130537 ... 0.16018583 0.23449521 0.19446358]
 [0.9559829  0.08788846 0.40750834 ... 0.         0.5147052  0.2934082 ]
 [1.039003   0.37614977 0.8492476  ... 0.29744506 0.5748082  0.08552614]
 ...
 [0.44141203 0.5637828  0.35702652 ... 0.24064513 0.46015424 0.39410967]
 [0.6171019  0.43130025 0.2638268  ... 0.24250628 0.42521703 0.39953476]
 [0.4126348  0.4872477  0.4075379  ... 0.3347038  0.58891547 0.6156447 ]]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Menghitung kesamaan cosine antar embedding
similarities = cosine_similarity(item_embeddings)
print("Matriks kesamaan:\n", similarities)

# Misalnya, mendapatkan 3 tempat wisata serupa untuk item pertama
similar_items = np.argsort(similarities[0])[-100:][::-1]  # Top 3 tempat wisata serupa
print("Tempat wisata serupa untuk item pertama:", similar_items)

for i in similar_items:
  if df['city_tag'][i] == df['city_tag'][0]:
    print(df['place_name'][i])


Matriks kesamaan:
 [[0.99999994 0.8071925  0.92740506 ... 0.8743285  0.85099375 0.8284715 ]
 [0.8071925  1.0000001  0.90960073 ... 0.79728633 0.8359204  0.75271994]
 [0.92740506 0.90960073 1.0000002  ... 0.82175016 0.82367545 0.7725476 ]
 ...
 [0.8743285  0.79728633 0.82175016 ... 1.0000001  0.9462949  0.94853985]
 [0.85099375 0.8359204  0.82367545 ... 0.9462949  0.9999998  0.9644752 ]
 [0.8284715  0.75271994 0.7725476  ... 0.94853985 0.9644752  0.9999998 ]]
Tempat wisata serupa untuk item pertama: [  0 135 129 477 506  16 428  83 529 457 181 490  19  59 308 216 507 372
 111 521 154 160 542 212 547 526 201   2 555 315 227 559 150  20 525 493
  33 537 528 469 407 565 478   5 482 550 551 545 139 512 487 162 563  60
 198 541 311 485 335 209 306 390 359 221 355 562 190 276 300 177 546  38
 263 291 226 556  65 498 317 481 172 218 222 356 267 518 508 152 170 387
 486 408 461 492 261 519  45 117 178 431]
Taman Mini Indonesia Indah
Taman Tangkuban Perahu
Atlantis Water Adventures Ancol
Pantai 

In [ ]:
model.save('cb_recommendation_model.h5')